In [1]:
import os
import json
import glob
from ultralytics import YOLO
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

# Initial Training

In [ ]:
# Train the model
os.chdir("/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_combined_train_dataset")
project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
dataset="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_combined_train_dataset"
batchsizes={ "11n": {3840: 1, 2880: 2, 1440: 4,  736: 16}, 
             "11s": {3840: 1, 2880: 1, 1440: 2,  736: 8},
             "11m": {3840: 1, 2880: 1, 1440: 2,  736: 5}, 
             "11l": {3840: 1, 2880: 1, 1440: 3,  736: 4}, 
           }
for model_name in ["11n", "11s"]: # "11n", "11s", "11m" ]:
    for imgsize in [2880, 1440]: #, 38402880]:# [736, 1440, 2880]:
        if imgsize == 3840 and model_name == "11s":
            continue
            
        for i in [150]: #list(range(1, 25)) + list(range(25, 50, 5)) + list(range(60, 101, 10)):
            name = f"{model_name}_{imgsize}imgsz_combined_{i}epochs"      
            weight_path = os.path.join(project, name, "weights", "epoch")
            completed_epochs = glob.glob(f"{weight_path}*.pt")
            resume_flag = False

            # if we started, see if we finished, if not resume
            if os.path.exists(os.path.join(f"{project}/{name}/weights/last.pt")):
                if len(completed_epochs) >= i:
                    print("We have {completed_epochs} pt files in {project}/{name}/weights, continuing")
                    continue
                    
                print(f"Resuming training from last.pt, {len(completed_epochs)} epochs completed previously")
                model = YOLO(f"{project}/{name}/weights/last.pt") 
                resume_flag = True
                print(f"Resuming {project}/{name} model")

            # starting fresh
            else:
                print(f"Loading pre-trained base model to start {project}/{name}")
                model = YOLO(f"yolo{model_name}.pt")  # load a pretrained model (recommended for training)
            
            results = model.train(resume=resume_flag, data=f"{dataset}/data.yaml", \
                                  project=project, name=name, epochs=i, imgsz=imgsize, batch=batchsizes[model_name][imgsize], 
                                  plots=True, val=False, save_period=1, save_txt=True)


Loading pre-trained base model to start /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11n_2880imgsz_combined_150epochs
New https://pypi.org/project/ultralytics/8.3.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.4.0 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_combined_train_dataset/data.yaml, epochs=150, time=None, patience=100, batch=2, imgsz=2880, save=True, save_period=1, cache=False, device=None, workers=8, project=/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative, name=11n_2880imgsz_combined_150epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, 

train: Scanning train/labels.cache... 1124 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1124/1124 [00:00<?, ?it/s]

train: WARNING ⚠️ train/images/IP8M-H-SW_2024-10-24_deer_00000111.png: 1 duplicate labels removed
train: WARNING ⚠️ train/images/IP8M-H-SW_2024-10-24_deer_00000112.png: 1 duplicate labels removed



val: Scanning val/labels.cache... 283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<?, ?it/s]


Plotting labels to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11n_2880imgsz_combined_150epochs/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 2880 train, 2880 val
Using 8 dataloader workers
Logging results to /data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative/11n_2880imgsz_combined_150epochs
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     87/150      9.29G      1.027     0.5615     0.9759          4       2880: 100%|██████████| 562/562 [02:02<00:00,  4.60it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     88/150      9.29G      1.033     0.5738     0.9609          9       2880: 100%|██████████| 562/562 [02:02<00:00,  4.61it/s]



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     89/150      8.76G      1.009     0.5687     0.9827          3       2880:  28%|██▊       | 158/562 [00:34<01:31,  4.40it/s]